## 『本次練習內容』
#### 使用Xception backbone做 Trnasfer Learning


## 『本次練習目的』
  #### 了解如何使用Transfer Learning
  #### 了解Transfer Learning的優點，可以觀察模型收斂速度

##### 可以自行嘗試多種架構

In [1]:
%tensorflow_version 2.x
import os
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras import backend as K
# from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Convolution2D, BatchNormalization
from tensorflow.keras.layers import MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
 
from tensorflow.keras.datasets import cifar10
import numpy as np
from sklearn.preprocessing import OneHotEncoder

TensorFlow 2.x selected.


In [2]:
input_tensor = Input(shape=(32, 32, 3))
#include top 決定要不要加入 fully Connected Layer

'''Xception 架構'''
model = tensorflow.keras.applications.xception.Xception(include_top=False, weights='imagenet',
                                                        input_tensor=input_tensor,
                                                        pooling=None, classes=1000)
  
'''Resnet 50 架構'''
# model = tensorflow.keras.applications.ResNet50(include_top=False, weights='imagenet',
#                                                input_tensor=input_tensor,
#                                                pooling=None, classes=10)
  
model.summary()

83689472/83683744 [==============================] - 7s 0us/step
Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 15, 15, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 15, 15, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 15, 15, 32)   0           block1_conv1_bn[0][0]            
__________________________

## 添加層數

In [3]:
x = model.output
  
'''可以參考Cifar10實作章節'''
# BatchNormalization皆不使用時，Model深度為139
x = Convolution2D(32, (1,1), activation='relu')(x)
x = BatchNormalization()(x)
x = Convolution2D(64, (1,1))(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(1,1))(x)
x = GlobalAveragePooling2D()(x)
x = Flatten()(x)
x = Dense(units=100, activation='relu')(x)

predictions = Dense(units=10, activation='softmax')(x)
model = Model(inputs=model.input, outputs=predictions)
  
print('Model深度：', len(model.layers))

Model深度： 141


## 鎖定特定幾層不要更新權重

In [4]:
for layer in model.layers[:100]:
  layer.trainable = False
    
for layer in model.layers[100:]:
  layer.trainable = True

## 準備 Cifar 10 資料

In [5]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train, X_test):
    mean = np.mean(X_train, axis=(0, 1, 2, 3))
    std = np.std(X_train, axis=(0, 1, 2, 3))
    X_train = (X_train - mean) / (std + 1e-7)
    X_test = (X_test - mean) / (std + 1e-7)
    
    return X_train, X_test    
    
## Normalize Training and Testset    
x_train, x_test = normalize(x_train, x_test) 

## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot = OneHotEncoder(categories='auto')
y_train = one_hot.fit_transform(y_train).toarray()
y_test = one_hot.transform(y_test).toarray()

170500096/170498071 [==============================] - 13s 0us/step
(50000, 32, 32, 3)


## Training

In [6]:
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ['accuracy'])
model.fit(x_train, y_train, batch_size=32, epochs=100)

Train on 50000 samples
Epoch 1/100
50000/50000 [==============================] - 39s 771us/sample - loss: 1.6670 - accuracy: 0.3928
Epoch 2/100
50000/50000 [==============================] - 28s 560us/sample - loss: 1.1730 - accuracy: 0.6016
Epoch 3/100
50000/50000 [==============================] - 28s 567us/sample - loss: 1.0184 - accuracy: 0.6525
Epoch 4/100
50000/50000 [==============================] - 29s 571us/sample - loss: 0.9106 - accuracy: 0.6915
Epoch 5/100
50000/50000 [==============================] - 29s 577us/sample - loss: 0.8318 - accuracy: 0.7163
Epoch 6/100
50000/50000 [==============================] - 29s 579us/sample - loss: 0.7514 - accuracy: 0.7438
Epoch 7/100
50000/50000 [==============================] - 29s 582us/sample - loss: 0.6693 - accuracy: 0.7718
Epoch 8/100
50000/50000 [==============================] - 29s 575us/sample - loss: 0.6060 - accuracy: 0.7910
Epoch 9/100
50000/50000 [==============================] - 29s 579us/sample - loss: 0.5396 - accu